# My first Variational Autoencoder for Jet compression

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot
import awkward

In [2]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

## Getting the data
I will only use $p_T, \eta, \phi \text{ and } E$.

In [3]:
path_to_data = '../../data/'

folder15 = 'breynold/user.breynold.data15_13TeV.00284484.physics_Main.DAOD_NTUP_JTRIG_JETM1.r9264_p3083_p3601_j042_tree.root/'
file15 = 'user.breynold.18753218._000001.tree.root'
folder16 = 'breynold/user.breynold.data16_13TeV.00307656.physics_Main.DAOD_NTUP_JTRIG_JETM1.r9264_p3083_p3601_j042_tree.root/'
file16 = 'user.breynold.18797259._000001.tree.root'

# Load a ROOT file
filePath = path_to_data + folder16 + file16
ttree = uproot.open(filePath)['outTree']['nominal']

In [4]:
branchnames = ['nAntiKt4EMTopoJets_Calib2018',
               'AntiKt4EMTopoJets_Calib2018_E',
               'AntiKt4EMTopoJets_Calib2018_pt',
               'AntiKt4EMTopoJets_Calib2018_phi',
               'AntiKt4EMTopoJets_Calib2018_eta']

jaggedE = ttree.array(branchnames[1])
jaggedpT = ttree.array(branchnames[2])
jaggedphi = ttree.array(branchnames[3])
jaggedeta = ttree.array(branchnames[4])

In [5]:
jaggedE.counts

array([20,  9,  9, ...,  5, 18, 11])

In [6]:
jaggedE.content

array([212.2173  , 175.15614 ,  64.451935, ...,  24.369474,  13.941364,
        60.927433], dtype=float32)

In [7]:
def get_leading(jaggedX):
    return jaggedX[jaggedX.counts > 1, 0]

In [8]:
leading_E = get_leading(jaggedE)
leading_pT = get_leading(jaggedpT)
leading_phi = get_leading(jaggedphi)
leading_eta = get_leading(jaggedeta)

In [9]:
print(leading_E.shape, leading_eta.shape, leading_phi.shape, leading_pT.shape)

(1937902,) (1937902,) (1937902,) (1937902,)


In [10]:
df = pd.DataFrame(data = {'pT': leading_pT, 'eta': leading_eta, 'phi': leading_phi, 'E': leading_E})

In [11]:
df.head(10)

,pT,eta,phi,E
0,161.850494,-0.764774,2.287350,212.217300
1,164.702454,0.205651,-1.074816,169.021805
2,405.421387,-0.064094,-2.324020,407.661316
3,139.671234,-0.289339,-2.052494,145.860703
4,227.195618,2.158644,-1.864455,996.913025
5,180.480667,1.684722,-1.049987,503.476410
6,39.454185,-0.305395,0.755678,41.943340
7,202.920883,1.583752,-0.301926,515.302856
8,263.867432,-0.141594,-2.308791,267.712372
9,234.460327,-2.949176,2.114750,2244.307617


## Splitting into training and test sets

In [12]:
n_features = len(df.loc[0])

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
print(train.shape, test.shape)

(1550321, 4) (387581, 4)


In [14]:
# Normalize the features
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
test = (test - train_mean) / train_std  # Is this the right way to normalize? (only using train mean and std to normalize both train and test)

train_x = train
test_x = test
train_y = train_x  # y = x since we are building and AE
test_y = test_x

train_ds = TensorDataset(torch.tensor(train_x.values), torch.tensor(train_y.values))
valid_ds = TensorDataset(torch.tensor(test_x.values), torch.tensor(test_y.values))

## Building the model

In [15]:
input_size = n_features
representation_size = input_size - 1
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.en1 = nn.Linear(input_size, 200)
        self.en_mu = nn.Linear(200, representation_size)
        self.en_std = nn.Linear(200, representation_size)
        self.de1 = nn.Linear(representation_size, 200)
        self.de2 = nn.Linear(200, input_size)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
    def encode(self, x):
        """Encode a batch of samples, and return posterior parameters for each point."""
        h1 = self.tanh(self.en1(x))
        return self.en_mu(h1), self.en_std(h1)
    
    def decode(self, z):
        """Decode a batch of latent variables"""
        
        h2 = self.tanh(self.de1(z))
        return self.sigmoid(self.de2(h2))
    
    def reparam(self, mu, logvar):
        """Reparameterisation trick to sample z values. 
        This is stochastic during training,  and returns the mode during evaluation."""
        
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
            
    
    def forward(self, x):
        """Takes a batch of samples, encodes them, and then decodes them again to compare."""
        mu, logvar = self.encode(x.view(-1, input_size))
        z = self.reparam(mu, logvar)
        return self.decode(z), mu, logvar
    
    def loss(self, reconstruction, x, mu, logvar):
        """ELBO assuming entries of x are binary variables, with closed form KLD."""
        
        bce = torch.nn.functional.binary_cross_entropy(reconstruction, x.view(-1, input_size))
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        # Normalise by same number of elements as in reconstruction
        KLD /= x.view(-1, input_size).data.shape[0] * input_size

        return bce + KLD
    
    def get_z(self, x):
        """Encode a batch of data points, x, into their z representations."""
        
        mu, logvar = self.encode(x.view(-1, input_size))
        return self.reparam(mu, logvar)

In [16]:
model = VAE().float()

In [17]:
# Some helper functions

def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )


def loss_batch(model, xb, yb, opt=None):
    #loss = loss_func(model(xb), yb)
    data = Variable(xb, requires_grad=False)
    reco_b, mu, logvar = model(data)
    loss = model.loss(reco_b, xb, mu, logvar)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)


def fit(epochs, model, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)  # MSE-Loss
        if(epoch % 1 == 0):
            print('epoch: ' + str(epoch), 'validation loss: ' + str(val_loss))

In [18]:
bs = 64  # batch size
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

## Training

In [19]:
epochs = 10
opt = optim.Adam(model.parameters(), lr=1e-3)
fit(epochs, model, opt, train_dl, valid_dl)

epoch: 0 validation loss: -24.151292958173794
epoch: 1 validation loss: -25.149526606664597
epoch: 2 validation loss: -24.622934093330315
epoch: 3 validation loss: -23.92693139149605
epoch: 4 validation loss: -23.53542617137865
epoch: 5 validation loss: -23.12029424828167
epoch: 6 validation loss: -23.107802431856054
epoch: 7 validation loss: -23.30426460938009
epoch: 8 validation loss: -23.612337796182757
epoch: 9 validation loss: -23.769608544127063


In [20]:
for ii in np.arange(100, 110):
    data = valid_ds.tensors[0][ii]
    pred = model(data)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

Inp: tensor([ 1.6863, -0.7095,  1.6946,  0.2721])
Out: (tensor([[1.0000e+00, 1.3746e-29, 1.0000e+00, 4.8075e-01]],
       grad_fn=<SigmoidBackward>), tensor([[-1.1429, -1.7562,  1.6886]], grad_fn=<AddmmBackward>), tensor([[-3.8708, -2.5482, -1.8221]], grad_fn=<AddmmBackward>))
 
Inp: tensor([ 0.1766, -0.1649, -0.0310, -0.5931])
Out: (tensor([[3.2607e-01, 4.3352e-19, 4.7836e-30, 3.6263e-33]],
       grad_fn=<SigmoidBackward>), tensor([[-0.3848, -0.6228, -0.6564]], grad_fn=<AddmmBackward>), tensor([[-3.1517, -3.5142, -1.4858]], grad_fn=<AddmmBackward>))
 
Inp: tensor([-0.3746,  0.6508,  1.3037, -0.5510])
Out: (tensor([[0.0000e+00, 5.1504e-01, 1.0000e+00, 7.8305e-33]],
       grad_fn=<SigmoidBackward>), tensor([[0.4120, 0.8205, 1.3976]], grad_fn=<AddmmBackward>), tensor([[-3.6306, -1.6341, -2.3438]], grad_fn=<AddmmBackward>))
 
Inp: tensor([-0.3711, -0.0072,  1.0894, -0.7293])
Out: (tensor([[0.0000e+00, 1.0737e-18, 1.0000e+00, 1.1845e-32]],
       grad_fn=<SigmoidBackward>), tensor([[-0.2